In [1]:
using SparseArrays
using LinearAlgebra
using Plots
using Statistics
using SuiteSparse
using VectorizedRoutines # for meshgrid

In [5]:
function lgwt(N,a,b)
    # lgwt.m This script is for computing definite integrals using Legendre-Gauss Quadrature.Computes the Legendre - Gauss nodes and weights on an interval [a, b] with truncation order N
    # Suppose you have a continuous function f(x) which is defined on[a, b]  which you can evaluate at any x in [a, b].Simply evaluate itat all of the values contained in the x vector to obtain a vector f.Then compute the definite integral using sum(f. * w);
    # Written by Greg von Winckel - 02 / 25 / 2004
    N = N - 1
    N1 = N + 1
    N2 = N + 2
    xu = range(-1,stop =  1, length =N1)
    xu = xu'

    # Initial guess
    y = cos.((2 * (0:N)'.+1)*pi/(2*N+2)).+(0.27/N1)*sin.(pi*xu*N/N2)
    # Legendre-Gauss Vandermonde Matrix
    L=zeros(N1, N2) # vu l'algo on pourrait avoir juste 2 variables vectorielles lk-1 et lk-2 pour moins de stockage

    # Compute the zeros of the N + 1 Legendre Polynomial using the recursion relation and the Newton - Raphson method
    y0 = 2
    eps = 2.2204e-16
    Lp = 0
    # Iterate until new points are uniformly within epsilon of old points
    while maximum(abs.(y .- y0)) > eps
        L[:, 1].=1
        L[:, 2]=y
        for k = 3:N2
            L[:, k]=((2*(k-1) -1 ) * y[:] .* L[:, k-1] - (k-2) * L[:, k-2] )/ (k-1) ; 
        end
        Lp = N2 * (L[:, N1] - y[:] .* L[:, N2])./ (1 .- y[:].^2);
        y0 = y
        y = y0[:] .- L[:, N2]./ Lp[:];
    end
    # Linear map from [-1, 1] to[a, b]
    x = (a * (1 .- y) + b * (1 .+ y)) / 2
    # Compute the weights
    w = (b - a) / ((1 .- y[:].^2) .* Lp[:].^2) * (N2 / N1)^2
    return x, w 
end

lgwt (generic function with 1 method)

In [19]:
function Wgp(x,y,Xc)
#  Evaluate characteristic function in each Gauss point
    End = length(Xc)
    ii=1:length(x);   
    X=Xc[1:6:End];
    Y=Xc[2:6:End];
    L=Xc[3:6:End];
    h=Xc[4:6:End];
    T=Xc[5:6:End];
    jj=1:length(X);
    I,J=Matlab.meshgrid(ii,jj)
    xi=reshape(x[I],size(I));
    yi=reshape(y[I],size(I));
    rho=sqrt.((X[J]-xi).^2+(Y[J]-yi).^2);
    drho_dX=(X[J]-xi)./(rho.+(rho==0));
    drho_dY=(Y[J]-yi)./(rho.+(rho==0));
    phi=atan.(-Y[J]+yi,-(X[J]-xi))-T[J]; 
    dphi_dX=((-Y[J]+yi)./(rho.^2 .+(rho==0)));
    dphi_dY=(X[J]-xi)./(rho.^2 .+(rho==0));
    dphi_dT=-ones(size(J));
    upsi=sqrt.(rho.^2+L[J].^2/4-rho.*L[J].*abs.(cos.(phi))).*(((rho.*cos.(phi)).^2)>=(L[J].^2/4))+~(((rho.*cos.(phi)).^2)>=(L[J].^2/4)).*abs.(rho.*sin.(phi));
    dupsi_drho=(2*rho-L[J].*abs.(cos.(phi)))/2 ./(upsi+(upsi==0)).*((((rho.*cos.(phi)).^2)>=(L[J].^2/4)))+~(((rho.*cos.(phi)).^2)>=(L[J].^2/4)).*abs.(sin.(phi));
    dupsi_dphi=(L[J].*rho.*sign(cos.(phi)).*sin.(phi))/2 ./(upsi+(upsi==0)).*((((rho.*cos.(phi)).^2)>=(L[J].^2/4)))+~(((rho.*cos.(phi)).^2)>=(L[J].^2/4)).*rho.*sign(sin.(phi)).*cos.(phi);
    dupsi_dL=(L[J]/2-rho.*abs.(cos.(phi)))./2 ./(upsi+(upsi==0)).*((((rho.*cos.(phi)).^2)>=(L[J].^2/4))&reshape([ i != 0 for i in upsi],size(a)));
    if method =="MMC"
        chi0=1-(4*upsi.^2 ./h[J].^2).^alp;
        dchi0_dh=8*alp*upsi.^2 .*(4*upsi.^2 ./h[J].^2).^(alp-1)./h.^3;
        dchi0_dupsi=-8*alp*upsi.*(4*upsi.^2 ./h[J].^2).^(alp-1)./h.^2;
        chi,dchi=Aggregation_Pi(chi0,p); 
        dchi_dh=(dchi0_dh.*dchi);
        dchi_dupsi=(dchi0_dupsi.*dchi);
        chi[chi<=-1e6]=-1e6;#seuil min
        W=(chi>epsi)+(chi<=epsi&chi>=-epsi).*(3/4*(1-bet)*(chi/epsi-chi.^3/3/epsi^3)+(1+bet)/2)+(chi<-epsi)*bet;
        dW_dchi=-3/4*(1/epsi-chi.^2/epsi^3).*(bet-1).*(abs(chi)<epsi);
        dW_dupsi=repmat(dW_dchi,size(dchi_dh,1),1).*dchi_dupsi;
        dW_dh=repmat(dW_dchi,size(dchi_dh,1),1).*dchi_dh;
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    elseif method == "GP"

        zetavar=upsi-h[J]/2;
        dzetavar_dupsi=ones(size(upsi));
        dzetavar_dh=-0.5*ones(size(J));
        deltaiel=(1/pi/r^2*(r^2*acos(zetavar/r)-zetavar.*sqrt(r^2-zetavar.^2))).*(abs(zetavar)<=r)+((zetavar<-r));
        ddetlaiel_dzetavar=(-2*sqrt(r^2-zetavar.^2)/pi/r^2).*(abs(zetavar)<=r);
        W=deltamin+(1-deltamin)*deltaiel;
        dW_ddeltaiel=(1-deltamin);
        dW_dh=dW_ddeltaiel*ddetlaiel_dzetavar.*dzetavar_dh;
        dW_dupsi=dW_ddeltaiel*ddetlaiel_dzetavar.*dzetavar_dupsi;
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    elseif method == "MNA"
        ds=upsi;
        d=abs(upsi);
        l=h(J)/2-epsi/2;
        u=h(J)/2+epsi/2;
        a3= -2 ./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a2=   (3*(l + u))./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a1=    -(6*l.*u)./((l - u).*(l.^2 - 2*l.*u + u.^2));
        a0=(u.*(- u.^2 + 3*l.*u))./((l - u).*(l.^2 - 2*l.*u + u.^2));
        W=1*(d<=l)+(a3.*d.^3+a2 .*d.^2+a1.*d+a0).*(d<=u&d>l);
        dW_dupsi=sign(ds).*(3*a3.*d.^2+2*a2 .*d+a1).*(d<=u&d>l);
        da3_du=- 2 ./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (2*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da2_du=3 ./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (3*(l + u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (3*(l + u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da1_du=- (6*l)./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (6*l.*u)./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (6*l.*u.*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da0_du=(- u.^2 + 3*l.*u)./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (u.*(- u.^2 + 3*l.*u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (u.*(3*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (u.*(- u.^2 + 3*l.*u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        dWf_du=(da3_du.*d.^3+da2_du.*d.^2+da1_du.*d+da0_du).*(d<=u&d>l);
        da3_dl=2 ./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) + (2*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da2_dl= 3 ./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (3*(l + u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (3*(l + u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da1_dl=    (6*l.*u)./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (6*u)./((l - u).*(l.^2 - 2*l.*u + u.^2)) + (6*l.*u.*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        da0_dl= (3*u.^2)./((l - u).*(l.^2 - 2*l.*u + u.^2)) - (u.*(- u.^2 + 3*l.*u))./((l - u).^2 .*(l.^2 - 2*l.*u + u.^2)) - (u.*(- u.^2 + 3*l.*u).*(2*l - 2*u))./((l - u).*(l.^2 - 2*l.*u + u.^2).^2);
        dWf_dl=(da3_dl.*d.^3+da2_dl.*d.^2+da1_dl.*d+da0_dl).*(d<=u&d>l);
        dW_dh=0.5*sign(ds).*(dWf_du+dWf_dl);
        dW_dX=dW_dupsi.*(dupsi_dphi.*dphi_dX+dupsi_drho.*drho_dX);
        dW_dY=dW_dupsi.*(dupsi_dphi.*dphi_dY+dupsi_drho.*drho_dY);
        dW_dL=dW_dupsi.*dupsi_dL;
        dW_dT=dW_dupsi.*dupsi_dphi.*dphi_dT;
    end
    return W,dW_dX,dW_dY,dW_dT,dW_dL,dW_dh
end
phi=Wgp(1:10,1:10,1:6)

MethodError: MethodError: no method matching isless(::Array{Float64,2}, ::Array{Float64,2})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
Closest candidates are:
  isless(!Matched::Missing, ::Any) at missing.jl:87
  isless(::Any, !Matched::Missing) at missing.jl:88

In [5]:
a = ones(3,3)
b = Matrix(I,3,3)
println(a,b)
reshape([ a[i]!= b[i] for i in 1:length(a)],size(a))

[1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]Bool[1 0 0; 0 1 0; 0 0 1]


3×3 Array{Bool,2}:
 0  1  1
 1  0  1
 1  1  0

In [15]:
function Aggregation_Pi(z)#penser a mettre en arg les P.qqchose et pour ceux qui ne sont pas dasn ttes les méthodes les mettre en =0 de base
# function that make the aggregation of the values in z and also compute sensitivities
    zm=repmat(max(z),size(z,1),1);
    #ka=p.ka;
    if aggregation == "asymptotic"
        Wa=sum(z);
        dWa=ones(size(z));
    elseif aggregation == "boolean"
        Wa=1-prod(1-z);
        dWa=repmat(prod(1-z.*reshape([ i != 1 for i in z],size(z))),size(z,1),1)./(1-z.*reshape([ i != 1 for i in z],size(z)));
    
    elseif aggregation == "p-norm"
        #zp=p.zp;
        zm=zm+zp;
        z=z+zp;
        Wa=exp(zm[1,:]).*(sum((z./exp(zm)).^ka,1)).^(1/ka)-zp;
        dWa=(z./exp(zm)).^(ka-1).*repmat((sum((z./exp(zm)).^ka,1)).^(1/ka-1),size(z,1),1);
    elseif aggregation == "p-mean"
        #zp=p.zp;
        zm=zm+zp;
        z=z+zp;
        Wa=exp(zm[1,:]).*(mean((z./exp(zm)).^ka,1)).^(1/ka)-zp;
        dWa=1/size(z,1)^(1/ka)*(z./exp(zm)).^(ka-1).*repmat((sum((z./exp(zm)).^ka,1)).^(1/ka-1),size(z,1),1);
    elseif aggregation == "KS"
        Wa=zm[1,:]+1/ka*log(sum(exp(ka*(z-zm)),1));
        dWa=exp(ka*(z-zm))./repmat(sum(exp(ka*(z-zm)),1),size(z,1),1);
    elseif aggregation == "KSl"
        Wa=zm[1,:]+1/ka*log(mean(exp(ka*(z-zm)),1));
        dWa=exp(ka*(z-zm))./repmat(sum(exp(ka*(z-zm)),1),size(z,1),1);
    else
        Wa=sum(z.*exp(ka*(z-zm)))./sum(exp(ka*(z-zm)),1);
        dWa=((exp(ka*(z-zm))+ka*z.*exp(ka*(z-zm))).*repmat(sum(exp(ka*(z-zm)),1),size(z,1),1)-repmat(sum(z.*exp(ka*(z-zm)),1),size(z,1),1)*ka.*exp(ka*(z-zm)))./repmat(sum(exp(ka*(z-zm)),1).^2,size(z,1),1);
        
    end
    return Wa,dWa
end

Aggregation_Pi (generic function with 1 method)

In [ ]:
function model_updateM(delta,p,X)
#update the Young Modulus on the base of delta
    m=X[6:6:length(X)];
    nc=length(m);
    m=repmat(m[:],1,size(delta,2));#verif ca
    
    if method == "MMC"
        E=E0*delta;
        dE_ddelta=E0*ones(size(delta));
        dE_ddelta=repmat(dE_ddelta,size(m,1),1);
        dE_dm=0*m;
        
    elseif metho == "GP"
        hatdelta=delta.*m.^gammac;
        [E,dE_dhatdelta]=Aggregation_Pi(hatdelta,p);
        if saturation
            [E,ds]=smooth_sat(E,p,nc);
            dE_dhatdelta=ds.*dE_dhatdelta;
        end
        E=E.*E0;
        dhatdelta_ddelta=m.^gammac;
        dhatdelta_dm=gammac*delta.*m.^(gammac-1);
        dE_ddelta=E0*dhatdelta_ddelta.*dE_dhatdelta;
        dE_dm=E0*dE_dhatdelta.*dhatdelta_dm;
        
        else #MNA
        hatdelta=delta.*m.^gammac;
        [rho,drho_dhatdelta]=Aggregation_Pi(hatdelta,p);
        if saturation
            [rho,ds]=smooth_sat(rho,p,nc);
            drho_dhatdelta=ds.*drho_dhatdelta;
        end
        E=rho.^penalty.*(E0-Emin)+Emin;
        dhatdelta_ddelta=m.^gammac;
        dhatdelta_dm=gammac*delta.*m.^(gammac-1);
        dE_ddelta=penalty*(E0-Emin)*dhatdelta_ddelta.*drho_dhatdelta.*rho.^(penalty-1);
        dE_dm=penalty*(E0-Emin)*dhatdelta_dm.*drho_dhatdelta.*rho.^(penalty-1);
    end
    return E,dE_ddelta,dE_dm
end

In [112]:
function topggp(nelx,nely,settings , volfrac = 0.5,maxiteration = 25  )
    starting_guess = "Crosses"
    stopping_criteria="change"  #Stopping Criteria:  'change'  'kktnorm'
    if settings=="GGP"
        method="MMC"   #Method:   'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        alp=1          #parameter for MMC
        epsi=0.866     #parameter for MMC
        bet=1e-3       #parameter for MMC
        minh=1              # minimum height of a component
        aggregation="KSl"   #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true     #switch for saturation
        ncx=1           #number of components in the x direction
        ncy=1           #number of components in the y direction
        Ngp=2           #number of Gauss point per sampling window
        R=0.5          #radius of the sampling window (infty norm)
        initial_d=0.5    #component initial mass
    elseif settings=="MMC"
        method="MMC"  #'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        alp=1          #parameter for MMC
        epsi=0.866     #parameter for MMC
        bet=1e-3       #parameter for MMC
        minh=1              # minimum height of a component
        aggregation="KSl"  #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true    #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5    #radius of the sampling window (infty norm)
        initial_d=0.5         #component initial mass
    elseif settings=="MNA"
        method="MNA"    #'MMC'  'MNA'  'GP'
        q=1             #q=1
        zp=1           #parameter for p-norm/mean regularization
        minh=1              # minimum height of a component
        sigma=1        #parameter for MNA
        penalty=3      #parameter for MNA
        gammav=1       #parameter for GP
        gammac=3       #parameter for GP
        aggregation="KSl"    #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10           #parameter for the aggregation constant
        saturation=true     #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5               #radius of the sampling window (infty norm)
        initial_d=0.5       #component initial mass
    elseif settings=="GP"
        method="GP"    #'MMC'  'MNA'  'GP'
        q=1                 #q=1
        zp=1           #parameter for p-norm/mean regularization
        deltamin=1e-6  #parameter for GP
        r=0.5          #parameter for GP
        minh=1              # minimum height of a component
        gammav=1       #parameter for GP
        gammac=3       #parameter for GP
        aggregation="KSl"  #parameter for the aggregation function to be used
        #'IE'-Induced Exponential  'KS'-KS function  'KSl'-Lowerbound KS function 'p-norm'  'p-mean'
        ka=10          #parameter for the aggregation constant
        saturation=true   #switch for saturation
        ncx=1               #number of components in the x direction
        ncy=1               #number of components in the y direction
        Ngp=2               #number of Gauss point per sampling window
        R=0.5    #radius of the sampling window (infty norm)
        initial_d=0.5       #component initial mass
    else
        print("settings string should be a valid entry: GGP, MMC, MNA or GP") 
        return        
    end
    E=1
    nu=0.3
    E0 = 1;
    Emin = 1e-6
    A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12]
    A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6]
    B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4]
    B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2]
    KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11])

    nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx); #number of the nodes in columns
    edofVec = reshape(2*nodenrs[1:end-1,1:end-1].+1,nelx*nely,1) ;#1st dof of each element (x top left)
    edofMat = zeros(nelx*nely, 8); #every line i contains the 8 dof of the ith element
    noeudsvoisins = [0 1 2*nely.+[2 3 0 1] -2 -1];
    for i = 1:8
        for j = 1:nelx*nely
            edofMat[j,i]= edofVec[j]+ noeudsvoisins[i] ;
        end
    end
    iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);# line to build  K
    jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);# columns
    
        # Define the nodal coordinates
    x=1
    Yy=zeros((nelx+1)*(nely+1))
    Xx=zeros((nelx+1)*(nely+1))

    for i = 1:(nelx+1)
        for j = 1:(nely+1)
            Yy[x]= j
            Xx[x]= i
            x+=1
        end
    end
    Yy=(nely+1).-Yy
    Xx=Xx.-1

        # Compute the centroid coordinates
    xc = zeros(nelx * nely)
    yc = zeros(nelx * nely)
    for xi = 0:(nelx-1)
        xc[(xi*nely+1):(xi+1)*nely].=xi+0.5
    end
    for yi = 0:(nely-1)
        yc[(yi+1):nely:nelx*nely].=yi+0.5
    end
    yc = nely .- yc
              
    centroid_coordinate=zeros(nelx*nely,2)
    for i = 1:nelx*nely
        centroid_coordinate[i,1]= xc[i]
        centroid_coordinate[i,2]= yc[i]
    end
    #l= 2*nelx*nely #length(centroid_coordinate)
    a=-R
    b=R
    gpc,wc=lgwt(Ngp,a,b)

    gpcx,gpcy=Matlab.meshgrid(gpc,gpc) #gpcx de taille ngp^2 normalement
    gauss_w = wc' * wc
    gpcx = repeat( gpcx,nelx*nely)[:] #ici on diffère de matlab...influence ? on verra
    gpcy = repeat( gpcy,nelx*nely)[:]
    gauss_weight=repeat(gauss_w,2*nelx*nely)[:]
    cc=repeat(centroid_coordinate,Ngp^2)
    gauss_point = cc + vcat(gpcx', gpcy')'
    ugp = unique(gauss_point, dims = 1)#ajouter les indices des 1eres occur plus tard

    #design variables
    xp=range(minimum(Xx),stop=maximum(Xx),length=ncx+2)
    yp=range(minimum(Yy),stop=maximum(Yy),length=ncy+2); 
    xx,yy=Matlab.meshgrid(xp,yp);
    Xc=repeat(xx[:],2,1);
    Yc=repeat(yy[:],2,1); 
    Lc=2*sqrt.((nelx/(ncx+2))^2+(nely/(ncy+2))^2)*ones(size(Xc)); 
    OnE = ones(convert(Int64,ceil(length(Xc)/2)))
    Tc=atan.(nely/ncy,nelx/ncx)*[OnE;-OnE];# component orientation angle tetha
    hc=2*ones(length(Xc),1); # component h
    Mc=initial_d*ones(size(Xc)); # component mass (For MNA and GP)
    Xg=vcat(Xc',Yc',Lc',hc',Tc',Mc')[:]
    
    #bounds
    Xl=minimum(Xx.-1)*ones(size(Xc));Xu=maximum(Xx.+1)*ones(size(Xc));
    Yl=minimum(Yy.-1)*ones(size(Xc));Yu=maximum(Yy.+1)*ones(size(Xc));
    Ll=0*ones(size(Xc));Lu=sqrt(nelx^2+nely^2)*ones(size(Xc));
    hl=minh*ones(size(Xc));hu=sqrt(nelx^2+nely^2)*ones(size(Xc));
    Tl=-2*pi*ones(size(Xc));Tu=2*pi*ones(size(Xc));
    Ml=0*ones(size(Xc));Mu=ones(size(Xc));
    lower_bound=vcat(Xl',Yl',Ll',hl',Tl',Ml')[:]
    upper_bound=vcat(Xu',Yu',Lu',hu',Tu',Mu')[:]
    X=(Xg-lower_bound)./(upper_bound-lower_bound);
    
    #init MMA
    loop = 0;
    m = 1;
    n = length(X(:));
    epsimin = 0.0000001;
    eeen    = ones(n);
    eeem    = ones(m);
    zeron   = zeros(n);
    zerom   = zeros(m);
    xval    = X(:);
    xold1   = xval;
    xold2   = xval;
    xmin    = zeron;
    xmax    = eeen;
    low     = xmin;
    upp     = xmax;
    C       = 1000*eeem;
    d       = 0*eeem;
    a0      = 1;
    a       = zerom;
    outeriter = 0;
    maxoutit  = 2000;
    kkttol  =0.001;
    changetol=0.001;
    kktnorm = kkttol+10;
    outit = 0;
    change=1;
    
    #for plots
    tt=0:0.005:(2*pi);tt=repmat(tt,length(Xc),1);
    cc=cos(tt);ss=sin(tt);
    
    if stopping_criteria == "kktnorm"
        stop_cond=outit < maxoutit && kktnorm>kkttol;
    else
        stop_cond=outit < maxoutit &&change>changetol;
    end
    
    #design loop
    while  stop_cond
        outit   = outit+1;
        outeriter = outeriter+1;
        #Compute the smooth characteristic functions and gradients for each component 
        # on each sampling window Gauss point (Can support GPU)
        W,dW_dX,dW_dY,dW_dT,dW_dL,dW_dh=Wgp(ugp[:,1],ugp[:,2],Xg,p);# -> coder wgp
        #Compute local volume fractions and gradients using generalized projection
        # delta is for densities, deltac for Young modulus
    
    return 
end


topggp (generic function with 3 methods)

In [113]:
topggp(60,30, "GGP" )


In [91]:
# gives m with one occur of each line, and give the indexes of the first occur
function doublons(m)#not working well so far
    index = []
    m2 = []
    l = length(m[1])
    for i in 1:(size(m)[1])
        ligne = m[i,:]
        isin = false
        for j in m2
            println(j)
            if m2[j,:]==ligne
                isin = true
                #j = size(m2[1]) #break ?
            end
        end
        if !isin
            append!(m2,[ligne])
            append!(index,i)
        end
    end
    return m2,index
end

doublons (generic function with 1 method)

UndefVarError: UndefVarError: repmat not defined

In [105]:
x= [1 3 8]
y =zeros(3,3)
for i = 1:3
    y[i,:]=x
end
z = [i for i in y[:]]
e = [7 8 9]
m = hcat(x',e',e')


1×3 Array{Int64,2}:
 1  3  8